In [1]:
%load_ext nb_black

import pandas as pd

dfs = {
    "train": pd.read_csv("census-income.data", header=None),
    "test": pd.read_csv("census-income.test", header=None),
}

for name, df in dfs.items():
    print(f"{name}: {df.shape}")

train: (199523, 42)
test: (99762, 41)


<IPython.core.display.Javascript object>

In [2]:
column_headers = [
    "age",
    "class of worker",
    "detailed industry recode",
    "detailed occupation recode",
    "education",
    "wage per hour",
    "enroll in edu inst last wk",
    "marital stat",
    "major industry code",
    "major occupation code",
    "race",
    "hispanic origin",
    "sex",
    "member of a labor union",
    "reason for unemployment",
    "full or part time employment stat",
    "capital gains",
    "capital losses",
    "dividends from stocks",
    "tax filer stat",
    "region of previous residence",
    "state of previous residence",
    "detailed household and family stat",
    "detailed household summary in household",
    "_instance weight",
    "migration code-change in msa",
    "migration code-change in reg",
    "migration code-move within reg",
    "live in this house 1 year ago",
    "migration prev res in sunbelt",
    "num persons worked for employer",
    "family members under 18",
    "country of birth father",
    "country of birth mother",
    "country of birth self",
    "citizenship",
    "own business or self employed",
    "fill inc questionnaire for veteran's admin",
    "veterans benefits",
    "weeks worked in year",
    "year",
]
dfs["test"].columns = column_headers
dfs["train"].columns = column_headers + ["Y"]

for name, df in dfs.items():
    print(f"{name}: {df.head()}")

train:    age                  class of worker  detailed industry recode  \
0   73                  Not in universe                         0   
1   58   Self-employed-not incorporated                         4   
2   18                  Not in universe                         0   
3    9                  Not in universe                         0   
4   10                  Not in universe                         0   

   detailed occupation recode                    education  wage per hour  \
0                           0         High school graduate              0   
1                          34   Some college but no degree              0   
2                           0                   10th grade              0   
3                           0                     Children              0   
4                           0                     Children              0   

  enroll in edu inst last wk    marital stat           major industry code  \
0            Not in universe         

<IPython.core.display.Javascript object>

That looks better! 
First check for NA values.

In [3]:
for name, df in dfs.items():
    s = df.isnull().sum(axis=0)
    na_cols = [(i, c) for i, c in s.items() if c > 0]
    for na_col in na_cols:
        print(f"{name}/{na_col[0]}: {na_col[1]}")

test/hispanic origin: 405
test/state of previous residence: 330
test/migration code-change in msa: 49946
test/migration code-change in reg: 49946
test/migration code-move within reg: 49946
test/migration prev res in sunbelt: 49946
test/country of birth father: 3429
test/country of birth mother: 3072
test/country of birth self: 1764


<IPython.core.display.Javascript object>

No NA's in the training data that is good.
However some test data have NA's

We will create dummies for the categories, NA's will have 0 values for all categories in a feature.

In [4]:
dfs2 = {}
for name, df in dfs.items():
    cats = df.select_dtypes(["object"])

    df[cats.columns] = cats.apply(lambda x: x.str.strip())
    dfs2[name] = pd.get_dummies(df, prefix=cats.columns, columns=cats.columns)

for name, df in dfs2.items():
    print(f"{name}: {df.shape}")

train: (199523, 411)
test: (99762, 399)


<IPython.core.display.Javascript object>

Hmmm many different column sizes, let us see the differences

In [5]:
extra_cols = sorted(list(set(dfs2["train"].columns) - set(dfs2["test"].columns)))
extra_cols

['Y_- 50000.',
 'Y_50000+.',
 'country of birth father_?',
 'country of birth mother_?',
 'country of birth self_?',
 'detailed household and family stat_Grandchild <18 ever marr not in subfamily',
 'hispanic origin_NA',
 'migration code-change in msa_?',
 'migration code-change in reg_?',
 'migration code-move within reg_?',
 'migration prev res in sunbelt_?',
 'state of previous residence_?']

<IPython.core.display.Javascript object>

The Y columns make sense, the others can be removed

In [6]:
dfs2["train"].drop(columns=extra_cols[2:], inplace=True)
dfs2["train"].shape

(199523, 401)

<IPython.core.display.Javascript object>

Actually the Y column is a binary column so we will remove the column "Y_- 50000." as it is complemetary with the "Y_50000+.". The rename it to Y again.

In [7]:
dfs2["train"].drop(columns="Y_- 50000.", axis=1, inplace=True)
dfs2["train"].rename({"Y_50000+.": "Y"}, inplace=True, axis=1)
dfs2["train"]["Y"].head()

0    0
1    0
2    0
3    0
4    0
Name: Y, dtype: uint8

<IPython.core.display.Javascript object>

Fine, and remove ignorable "_instance weight" col

In [8]:
for name, df in dfs2.items():
    df.drop(columns="_instance weight", inplace=True, axis=1)
    print(f"{name}: {df.shape}")

train: (199523, 399)
test: (99762, 398)


<IPython.core.display.Javascript object>

And let us check for more NA like columns

In [9]:
dfs2["train"].info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199523 entries, 0 to 199522
Data columns (total 399 columns):
 #   Column                                                                              Dtype
---  ------                                                                              -----
 0   age                                                                                 int64
 1   detailed industry recode                                                            int64
 2   detailed occupation recode                                                          int64
 3   wage per hour                                                                       int64
 4   capital gains                                                                       int64
 5   capital losses                                                                      int64
 6   dividends from stocks                                                               int64
 7   num persons worked for emplo

<IPython.core.display.Javascript object>

Ok we have our basis data set now with each field category in a binary feature. The training set has no NA's but the test set has. That will result in features that have all zeroes for their field category flags. 

E.g. this table:

| # | colour |
|---|--------|
| 0 | red    |
| 1 | green  |
| 2 | NaN    |

Will become:

| # | colour_red | colour_green |
|---|------------|--------------|
| 0 | 1          | 0            |
| 1 | 0          | 1            |
| 2 | 0          | 0            |

Which is good for classification.

Next we create X and Y values for training.

In [10]:
X_train = dfs2["train"].drop("Y", axis=1)
y_train = dfs2["train"]["Y"]
X_test = dfs2["test"]

<IPython.core.display.Javascript object>

Now save these data sets

In [11]:
X_train.to_csv("x_train.csv", index_label="Index")
y_train.to_csv("y_train.csv", index_label="Index")
X_test.to_csv("x_test.csv", index_label="Index")

<IPython.core.display.Javascript object>